# Game Genres Notebook

In [ ]:
query = '''
SELECT * FROM game_genres ORDER BY game_name;
'''
df=pd_sql.read_sql(query, connection)

In [ ]:
print(df.columns)
df.shape

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df['game_genres'].unique()

In [ ]:
df.loc[df['game_genres'] == 'IRL']